In [1]:
# import libraries
import pandas as pd
import numpy as np

## Similaridad coseno
¿Cómo calcularla en Python?

Supongamos que tenemos la siguiente matriz:

|  	| Libro A 	| Libro B 	| Libro C 	|
|-------	|---------	|---------	|---------	|
| Juan 	| 5 	| 4 	| 4 	|
| Diego 	| 4 	| 5 	| 5 	|


Podemos calcular la similaridad  coseno empleando sklearn:

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
Juan = [5,4,4]
Diego = [4,5,5]
cosine_similarity([Juan, Diego])

array([[1.        , 0.97823198],
       [0.97823198, 1.        ]])

También podemos calcular la similaridad a mano:

In [3]:
(5*4 + 4*5 + 4*5)/(np.sqrt(5**2+4**2+4**2)*np.sqrt(4**2+5**2+5**2))

0.9782319760890369

O empleando Numpy

In [4]:
np.dot(Juan,Diego)/np.dot(np.linalg.norm(Juan), np.linalg.norm(Diego))

0.9782319760890369

Ahora bien, cuando tenemos una matriz user-item de la vida real, tenemos muchos casos faltantes. En esta situación, no podremos calcular la similaridad coseno tan fácilmente... 

In [5]:
user_item = np.array([[5, np.nan, 4],[4,3,5],[4,5,5],[np.nan, 5, np.nan], [np.nan, 5, 3]])

In [6]:
user_item

array([[ 5., nan,  4.],
       [ 4.,  3.,  5.],
       [ 4.,  5.,  5.],
       [nan,  5., nan],
       [nan,  5.,  3.]])

## Item-Based Collaborative Filtering

In [7]:
mlens = pd.read_csv("../Data/u.data",sep="\t",header=None)
mlens.columns = ["user_id","item_id","rating","timestamp"]

In [8]:
### Tomamos una muestra
mlens = mlens.sample(5000)

In [9]:
mlens.head()

,user_id,item_id,rating,timestamp
2920,307,1411,4,877124058
43437,25,404,3,885852920
20662,151,462,4,879524088
46833,679,121,2,884488260
55530,329,39,2,891656391


#### Construcción de la matriz de similaridad item-item
Antes de construir esta matriz necesitamos normalizar por item

#### Normalización

Para cada item, calculamos el score promedio y se lo restamos. Esto se hace para quitar el efecto propio de que una película sea "buena" o "mala". Por ejemplo, El Padrino 1 en general debería tener mejores rankings que El Padrino 3, sin embargo, ambas películas se deben asemejar a las mismas películas. 
Por este motivo, a cada item le restamos su media.

In [10]:
# Calculamos la media por item
rating_mean = mlens.groupby(['item_id'], as_index = False, sort = False).mean()
rating_mean = rating_mean.drop(["user_id","timestamp"],axis=1)
rating_mean.columns = ["item_id","mean_rating"]

In [11]:
# Juntamos con la matriz original
adjusted_ratings = pd.merge(mlens,rating_mean,on = 'item_id', how = 'left', sort = False)
adjusted_ratings['rating_adjusted']=adjusted_ratings['rating']-adjusted_ratings['mean_rating']
adjusted_ratings.head()

,user_id,item_id,rating,timestamp,mean_rating,rating_adjusted
0,307,1411,4,877124058,4.000000,0.000000
1,25,404,3,885852920,3.333333,-0.333333
2,151,462,4,879524088,3.916667,0.083333
3,679,121,2,884488260,3.413793,-1.413793
4,329,39,2,891656391,3.250000,-1.250000


In [12]:
# Reemplazamos los 0 por 1*e-8 para evitar problemas
adjusted_ratings.loc[adjusted_ratings['rating_adjusted'] == 0, 'rating_adjusted'] = 1e-8


In [13]:
adjusted_ratings

,user_id,item_id,rating,timestamp,mean_rating,rating_adjusted
0,307,1411,4,877124058,4.000000,1.000000e-08
1,25,404,3,885852920,3.333333,-3.333333e-01
2,151,462,4,879524088,3.916667,8.333333e-02
3,679,121,2,884488260,3.413793,-1.413793e+00
4,329,39,2,891656391,3.250000,-1.250000e+00
5,717,890,1,884642001,2.600000,-1.600000e+00
6,932,502,4,891249710,4.000000,1.000000e-08
7,286,1101,5,877532715,4.000000,1.000000e+00
8,712,228,3,874730261,3.625000,-6.250000e-01
9,267,515,5,878970710,4.250000,7.500000e-01


### Matriz de similaridad

A continuación se encuentra la función que permite generar la matriz de similaridad.

In [14]:
# Función para calcular la matriz item a item
def build_w_matrix(adjusted_ratings):
    # Inicializo matriz de pesos
    w_matrix_columns = ['movie_1', 'movie_2', 'weight']
    w_matrix=pd.DataFrame(columns=w_matrix_columns)
    
    # Me quedo con un array de películas distintas
    distinct_movies = np.unique(adjusted_ratings['item_id'])

    i = 0
    # Para cada movie_1 en las distintas películas
    for movie_1 in distinct_movies:

        if i%10==0:
            print(i , "out of ", len(distinct_movies))

        # Extraigo todos los usuarios que puntuaron movie_1
        user_data = adjusted_ratings[adjusted_ratings['item_id'] == movie_1]
        distinct_users = np.unique(user_data['user_id'])

        #### Guardo todos los rating de los usuarios que puntuaron tanto movie_1 como movie_2 #####
        record_row_columns = ['user_id', 'movie_1', 'movie_2', 'rating_adjusted_1', 'rating_adjusted_2']
        record_movie_1_2 = pd.DataFrame(columns=record_row_columns)
        
        # Para cada usuario C que puntuó movie_1
        for c_userid in distinct_users:
            # Rating del usuario C a movie_1
            c_movie_1_rating = user_data[user_data['user_id'] == c_userid]['rating_adjusted'].iloc[0]
            # Extraigo películas puntuadas por C excluyendo movie_1
            c_user_data = adjusted_ratings[(adjusted_ratings['user_id'] == c_userid) & (adjusted_ratings['item_id'] != movie_1)]
            c_distinct_movies = np.unique(c_user_data['item_id'])

            # Para cada otra película (movie_2) que puntuó el usuario C
            for movie_2 in c_distinct_movies:
                # Obtengo el rating para movie_2 de C
                c_movie_2_rating = c_user_data[c_user_data['item_id'] == movie_2]['rating_adjusted'].iloc[0]
                record_row = pd.Series([c_userid, movie_1, movie_2, c_movie_1_rating, c_movie_2_rating], index=record_row_columns)
                record_movie_1_2 = record_movie_1_2.append(record_row, ignore_index=True)

        # Calculo la similaridad entre movie_1 y las demás películas guardadas
        distinct_movie_2 = np.unique(record_movie_1_2['movie_2'])
        # Para cada movie_2

        for movie_2 in distinct_movie_2:
            paired_movie_1_2 = record_movie_1_2[record_movie_1_2['movie_2'] == movie_2]
            sim_value_numerator = (paired_movie_1_2['rating_adjusted_1'] * paired_movie_1_2['rating_adjusted_2']).sum()
            sim_value_denominator = np.sqrt(np.square(paired_movie_1_2['rating_adjusted_1']).sum()) * np.sqrt(np.square(paired_movie_1_2['rating_adjusted_2']).sum())
            sim_value_denominator = sim_value_denominator if sim_value_denominator != 0 else 1e-8
            sim_value = sim_value_numerator / sim_value_denominator
            w_matrix = w_matrix.append(pd.Series([movie_1, movie_2, sim_value], index=w_matrix_columns), ignore_index=True)

        i = i + 1

    return w_matrix

In [15]:
# Construimos la matriz de similitudes
%time w_matrix = build_w_matrix(adjusted_ratings)

0 out of  1044
10 out of  1044
20 out of  1044
30 out of  1044
40 out of  1044
50 out of  1044
60 out of  1044
70 out of  1044
80 out of  1044
90 out of  1044
100 out of  1044
110 out of  1044
120 out of  1044
130 out of  1044
140 out of  1044
150 out of  1044
160 out of  1044
170 out of  1044
180 out of  1044
190 out of  1044
200 out of  1044
210 out of  1044
220 out of  1044
230 out of  1044
240 out of  1044
250 out of  1044
260 out of  1044
270 out of  1044
280 out of  1044
290 out of  1044
300 out of  1044
310 out of  1044
320 out of  1044
330 out of  1044
340 out of  1044
350 out of  1044
360 out of  1044
370 out of  1044
380 out of  1044
390 out of  1044
400 out of  1044
410 out of  1044
420 out of  1044
430 out of  1044
440 out of  1044
450 out of  1044
460 out of  1044
470 out of  1044
480 out of  1044
490 out of  1044
500 out of  1044
510 out of  1044
520 out of  1044
530 out of  1044
540 out of  1044
550 out of  1044
560 out of  1044
570 out of  1044
580 out of  1044
590 out 

### Predecir ratings

In [16]:
# Predecir ratings
def predict(user_id, item_id, w_matrix, adjusted_ratings, rating_mean):
    
    # Si  está calculado el mean_rating
    if rating_mean[rating_mean['item_id'] == item_id].shape[0] > 0:
        mean_rating = rating_mean[rating_mean['item_id'] == item_id]['mean_rating'].iloc[0]
    # Si no está calculado el mean_rating, porque no estaba puntuada la película
    else:
        mean_rating = 2.5

    #### Calcular el rating para una determinada película para un usuario ####
    
    # Ratings del usuario
    user_other_ratings = adjusted_ratings[adjusted_ratings['user_id'] == user_id]
    
    # Items que el usuario puntuó
    user_distinct_movies = np.unique(user_other_ratings['item_id'])
    sum_weighted_other_ratings = 0
    sum_weghts = 0
    
    # Itero por cada película que puntuó
    for movie_j in user_distinct_movies:
        if rating_mean[rating_mean['item_id'] == movie_j].shape[0] > 0:
            # Rating medio para la película j
            rating_mean_j = rating_mean[rating_mean['item_id'] == movie_j]['mean_rating'].iloc[0]
        else:
            # Si el usuario no puntuó nada
            rating_mean_j = 2.5
            
        # Sólo calculamos los valores pesados cuando la película a predecir y la película j están en la matriz de similitudes
        w_movie_1_2 = w_matrix[(w_matrix['movie_1'] == item_id) & (w_matrix['movie_2'] == movie_j)]
        if w_movie_1_2.shape[0] > 0:
            # Rating que el usuario asignó a j
            user_rating_j = user_other_ratings[user_other_ratings['item_id']==movie_j]
            # Resto el promedio del item a la predicción y lo multiplico por el peso, y agrego ésto
            sum_weighted_other_ratings += (user_rating_j['rating'].iloc[0] - rating_mean_j) * w_movie_1_2['weight'].iloc[0]
            # Agreo el valor absoluto del peso
            sum_weghts += np.abs(w_movie_1_2['weight'].iloc[0])

    # Si la suma de pesos es 0
    if sum_weghts == 0:
        predicted_rating = mean_rating
    # Si la suma de pesos es mayor a 0
    else:
        predicted_rating = mean_rating + sum_weighted_other_ratings/sum_weghts

    return predicted_rating

In [17]:
# Predecir un rating
predict(2, 132, w_matrix, adjusted_ratings, rating_mean)

3.7